In [1]:
import numpy as np
import qutip as qt

In [2]:
def is_stabilized(state, stabilizers):
    for stabilizer in stabilizers:
        # print(state.overlap(stabilizer * state))
        if not state.overlap(stabilizer * state).real > 0.99:
            return False
    return True

def print_quantum_state_as_superposition(state):
    state = state.unit()
    dimension = int(np.log2(state.shape[0]))
    state_vector = state.full().flatten()
    basis_states = ['0', '1']
    
    superposition = []
    amp = 0
    for i, amplitude in enumerate(state_vector):
        if np.abs(amplitude) > 1e-10:
            binary_state = ''.join(basis_states[int(bit)] for bit in f"{i:0{dimension}b}")
            superposition.append(f"|{binary_state}⟩")
            amp = amplitude
    superposition_str = ' + '.join(superposition)
    
    print(f"|ψ⟩ = {np.real(amp):.2f}({superposition_str})")

Checking 3-qubit bitflip

In [3]:
# Define the Pauli matrices
I = qt.qeye(2)
X = qt.sigmax()
Y = qt.sigmay()
Z = qt.sigmaz()

# Define the stabilizer generators for the 3-qubit bit-flip code
stabilizers = [
    qt.tensor(I, Z, Z),
    qt.tensor(Z, I, Z)
]

# Define the basis states for three qubits
basis_states = [qt.basis(2, 0), qt.basis(2, 1)]

# Define the logical zero state |0_L>
logical_zero = (qt.tensor(basis_states[0], basis_states[0], basis_states[0])).unit()

# Define the logical one state |1_L>
logical_one = (qt.tensor(basis_states[1], basis_states[1], basis_states[1])).unit()

# Display the logical states
print_quantum_state_as_superposition(logical_zero)
print_quantum_state_as_superposition(logical_one)

# Check if the logical states are stabilized
print("Logical |0_L> is stabilized:", is_stabilized(logical_zero, stabilizers))
print("Logical |1_L> is stabilized:", is_stabilized(logical_one, stabilizers))

# Define the logical qubit as a superposition of |0_L> and |1_L>
alpha, beta = 1/np.sqrt(2), 1/np.sqrt(2)  # Example coefficients
logical_qubit = (alpha * logical_zero + beta * logical_one).unit()


print_quantum_state_as_superposition(logical_qubit)

|ψ⟩ = 1.00(|000⟩)
|ψ⟩ = 1.00(|111⟩)
Logical |0_L> is stabilized: True
Logical |1_L> is stabilized: True
|ψ⟩ = 0.71(|000⟩ + |111⟩)


Applying error to the 3 qubit repetition code and checking the syndromes

In [4]:
def is_stabilized(state, stabilizers):
    syndrome = []
    for stabilizer in stabilizers:
        print(np.real(state.overlap(stabilizer * state)))
        if not state.overlap(stabilizer * state).real > 0.99:
            syndrome.append(1)
        else:
            syndrome.append(0)
    return syndrome


# Apply error to the first bit
X1 = qt.tensor(X, I, I)
logical_state = X1 * logical_zero

print("Logical |0_L> is stabilized:", is_stabilized(logical_state, stabilizers))

1.0
-1.0
Logical |0_L> is stabilized: [0, 1]


Checking Steane code

In [74]:
# Define the Pauli matrices
I = qt.qeye(2)
X = qt.sigmax()
Y = qt.sigmay()
Z = qt.sigmaz()

# Define the stabilizer generators for the 3-qubit bit-flip code
stabilizers = [
    qt.tensor(Z, I, Z, I, Z, I, Z),
    qt.tensor(I, Z, Z, I, I, Z, Z),
    qt.tensor(I, I, I, Z, Z, Z, Z),
    qt.tensor(X, I, X, I, X, I, X),
    qt.tensor(I, X, X, I, I, X, X),
    qt.tensor(I, I, I, X, X, X, X)
]

# Define the basis states for three qubits
basis_states = [qt.basis(2, 0), qt.basis(2, 1)]

# Define the logical zero state |0_L>
logical_zero = (qt.tensor(basis_states[0], basis_states[0], basis_states[0], basis_states[0], basis_states[0], basis_states[0], basis_states[0]) + 
                qt.tensor(basis_states[1], basis_states[0], basis_states[1],basis_states[0], basis_states[1], basis_states[0], basis_states[1]) + 
                qt.tensor(basis_states[0], basis_states[1], basis_states[1],basis_states[0], basis_states[0], basis_states[1], basis_states[1]) + 
                qt.tensor(basis_states[1], basis_states[1], basis_states[0],basis_states[0], basis_states[1], basis_states[1], basis_states[0]) + 
                qt.tensor(basis_states[0], basis_states[0], basis_states[0],basis_states[1], basis_states[1], basis_states[1], basis_states[1]) + 
                qt.tensor(basis_states[1], basis_states[0], basis_states[1],basis_states[1], basis_states[0], basis_states[1], basis_states[0]) + 
                qt.tensor(basis_states[0], basis_states[1], basis_states[1],basis_states[1], basis_states[1], basis_states[0], basis_states[0]) + 
                qt.tensor(basis_states[1], basis_states[1], basis_states[0],basis_states[1], basis_states[0], basis_states[0], basis_states[1])
                ).unit()



# Define the logical one state |1_L>
X7 = qt.tensor(X, X, X, X, X, X, X)
logical_one = X7 * logical_zero

# Display the logical states
# print("Logical |0_L>:\n", logical_zero)
# print("Logical |1_L>:\n", logical_one)
# print_quantum_state_as_superposition(logical_zero)
print_quantum_state_as_superposition(logical_one)
print_quantum_state_as_superposition(logical_zero)


# Check if the logical states are stabilized
print("Logical |0_L> is stabilized:", is_stabilized(logical_zero, stabilizers))
print("Logical |1_L> is stabilized:", is_stabilized(logical_one, stabilizers))

# Define the logical qubit as a superposition of |0_L> and |1_L>
alpha, beta = 1/np.sqrt(2), 1/np.sqrt(2)  # Example coefficients
logical_qubit = (alpha * logical_zero + beta * logical_one).unit()

# print_quantum_state_as_superposition(logical_qubit)
# Display the logical qubit state
# print("Logical qubit:\n", logical_qubit)

|ψ⟩ = (0.35)|0010110⟩ + (0.35)|0011001⟩ + (0.35)|0100101⟩ + (0.35)|0101010⟩ + (0.35)|1000011⟩ + (0.35)|1001100⟩ + (0.35)|1110000⟩ + (0.35)|1111111⟩
|ψ⟩ = (0.35)|0000000⟩ + (0.35)|0001111⟩ + (0.35)|0110011⟩ + (0.35)|0111100⟩ + (0.35)|1010101⟩ + (0.35)|1011010⟩ + (0.35)|1100110⟩ + (0.35)|1101001⟩
(0.9999999999999999+0j)
(0.9999999999999999+0j)
(0.9999999999999999+0j)
(0.9999999999999999+0j)
(0.9999999999999999+0j)
(0.9999999999999999+0j)
Logical |0_L> is stabilized: []
(0.9999999999999999+0j)
(0.9999999999999999+0j)
(0.9999999999999999+0j)
(0.9999999999999999+0j)
(0.9999999999999999+0j)
(0.9999999999999999+0j)
Logical |1_L> is stabilized: []


In [86]:
def is_stabilized(state, stabilizers):
    syndrome = []
    for stabilizer in stabilizers:
        print(state.overlap(stabilizer * state))
        if not state.overlap(stabilizer * state).real > 0.99:
            syndrome.append(stabilizer)
    return syndrome

# Apply error to the first bit
X1 = qt.tensor(I, I, I, I, I, I, Z)
logical_state = X1 * logical_zero

print("Logical |0_L> is stabilized:", is_stabilized(logical_state, stabilizers))

(0.9999999999999999+0j)
(0.9999999999999999+0j)
(0.9999999999999999+0j)
(-0.9999999999999999+0j)
(-0.9999999999999999+0j)
(-0.9999999999999999+0j)
Logical |0_L> is stabilized: [Quantum object: dims=[[2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2]], shape=(128, 128), type='oper', dtype=CSR, isherm=True
Qobj data =
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], Quantum object: dims=[[2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2]], shape=(128, 128), type='oper', dtype=CSR, isherm=True
Qobj data =
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], Quantum object: dims=[[2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2]], shape=(128, 128), type='oper', dtype=CSR, isherm=True
Qobj data =
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]